In [24]:
import pandas as pd
import openpyxl
import sqlite3

sh_users = pd.read_excel('users.xlsx')
sh_users.sort_values(by="name", inplace=True)

conn = sqlite3.connect('users.db')
cursor = conn.cursor()

cursor.execute('SELECT * FROM users ORDER BY name ASC')
db_users = cursor.fetchall()

conn.commit()
conn.close()

column_names = [description[0] for description in cursor.description]
db_users = pd.DataFrame(db_users, columns=column_names)

sh_users.info(), db_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               2 non-null      object
 1   number             2 non-null      int64 
 2   email              2 non-null      object
 3   reporting_officer  2 non-null      object
 4   hod                2 non-null      object
dtypes: int64(1), object(4)
memory usage: 212.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               2 non-null      object
 1   number             2 non-null      int64 
 2   email              2 non-null      object
 3   reporting_officer  2 non-null      object
 4   hod                2 non-null      object
dtypes: int64(1), object(4)
memory usage: 212.0+ bytes


(None, None)

In [25]:
exact_match = sh_users.equals(db_users)

if not exact_match:
    # create 2 dataframes to compare
    old_users = pd.merge(sh_users, db_users, how="outer", indicator=True).query('_merge == "right_only"').drop(columns='_merge')
    new_users = pd.merge(sh_users, db_users, how="outer", indicator=True).query('_merge == "left_only"').drop(columns='_merge')

db_users

,name,number,email,reporting_officer,hod
0,Rachmiel Teo Ren Xiang,3333,rach@go.edu.sg,fong_chee_kiang@go.edu.sg,lim_zong_han@go.edu.sg
1,Shawn Tan Minyi,23456789,shawn@go.edu.sg,fong_chee_kiang@go.edu.sg,lim_zong_han@go.edu.sg


In [26]:
old_users

,name,number,email,reporting_officer,hod
2,Rachmiel Teo Ren Xiang,23456,rach@go.edu.sg,fong_chee_kiang@go.edu.sg,lim_zong_han@go.edu.sg


In [27]:
new_users

,name,number,email,reporting_officer,hod
0,Rachmiel Teo Ren Xiang,3333,rach@go.edu.sg,fong_chee_kiang@go.edu.sg,lim_zong_han@go.edu.sg


In [29]:
old_users_tuples = [tuple(old_user) for old_user in old_users.values]
new_users_tuples = [tuple(new_user) for new_user in new_users.values]

conn = sqlite3.connect('users.db')
cursor = conn.cursor()

for name, number, email, reporting_officer, hod in old_users_tuples:
    cursor.execute('DELETE FROM users WHERE name = ?', (name,))
    
for name, number, email, reporting_officer, hod in new_users_tuples:
    cursor.execute('INSERT INTO users (name, number, email, reporting_officer, hod) VALUES (?, ?, ?, ?, ?)', (name, number, email, reporting_officer, hod))

conn.commit()
conn.close()